In [2]:
# Required Libraries
import asyncio  # Library used for asynchronous programming.
import requests  # Library used for making HTTP requests.
import pandas as pd  # Library used for data analysis and table processing.
import gradio as gr  # Library used for creating user interfaces.
import cv2  # Library used for image processing.
import numpy as np  # Library used for mathematical operations.
from tensorflow.keras.models import load_model  # Library used to load pre-trained models.
from PIL import Image  # Library used for working with images.
from io import BytesIO  # Library used for handling binary data in memory.
import json  # Library used for processing JSON data.

# Loading Models and Classification Functions
# Load the pre-trained Continuum model.
continuum_cnn1 = load_model("continuum_cnn1.h5")
continuum_class_names = ['continuum-alpha', 'continuum-beta']

def predict_continuum_image(img):  # Predicts for a Continuum image.
    img_resized = cv2.resize(img, (256, 256))  # Resizes the image to fit the model.
    img_resized = img_resized / 255.0  # Normalization.
    prediction = continuum_cnn1.predict(np.array([img_resized]))[0]
    return {continuum_class_names[i]: float(prediction[i]) for i in range(len(continuum_class_names))}

# Load the pre-trained Magnetogram model.
magnetogram_cnn1 = load_model("magnetogram_cnn1.h5")
magnetogram_class_names = ['magnetogram-alpha', 'magnetogram-beta']

def predict_magnetogram_image(img):  # Predicts for a Magnetogram image.
    img_resized = cv2.resize(img, (256, 256))  # Resizes the image to fit the model.
    img_resized = img_resized / 255.0  # Normalization.
    prediction = magnetogram_cnn1.predict(np.array([img_resized]))[0]
    return {magnetogram_class_names[i]: float(prediction[i]) for i in range(len(magnetogram_class_names))}

# URL for Fetching KP Index Data and Classification Functions
KP_URL = "https://services.swpc.noaa.gov/json/planetary_k_index_1m.json"

# Function to classify the KP value and return an explanation.
def classify_kp(kp_value):
    if kp_value == 5:
        return "G1 (Minor): Small fluctuations may occur in power systems."
    elif kp_value == 6:
        return "G2 (Moderate): Voltage alarms may occur."
    elif kp_value == 7:
        return "G3 (Strong): Some protective devices may give false alarms."
    elif kp_value == 8:
        return "G4 (Severe): Widespread voltage control issues may occur."
    elif kp_value == 9:
        return "G5 (Extreme): Power outages and network failures may occur."
    else:
        return "Low likelihood of a storm."

async def get_kp_value():
    try:
        resp = requests.get(KP_URL, timeout=10)  # Connects to the URL; raises an error if data is not received within 10 seconds.
        resp.raise_for_status()  # Raises an error if the response is not valid.
        data = resp.json()  # Converts the response to JSON format.
        df = pd.DataFrame(data)  # Converts the JSON data into a DataFrame.
        kp_val = int(df['kp_index'].iloc[-1])  # Retrieves the latest KP index value from the DataFrame.
        return kp_val
    except Exception as e:
        return f"Error: {e}"  # Returns an error message if something goes wrong.

def fetch_kp_sync():  # Runs the asynchronous KP fetching function synchronously.
    return asyncio.run(get_kp_value())

# Functions for Fetching and Cropping Image Data from a URL
# URLs for Continuum and Magnetogram data
CONTINUUM_URL = "https://sdo.gsfc.nasa.gov/assets/img/latest/latest_4096_HMIIC.jpg"
MAGNETOGRAM_URL = "https://sdo.gsfc.nasa.gov/assets/img/latest/latest_4096_HMIB.jpg"

# If images can be fetched from the URL, they are cropped; otherwise, an error is raised.
def fetch_and_crop_image(url):  # Fetches and crops the image from the given URL.
    try:
        response = requests.get(url)
        response.raise_for_status()  # Checks for request errors.
        img = Image.open(BytesIO(response.content))
        img_np = np.array(img)
        # Crops the image to use only the equatorial region of the sun.
        if len(img_np.shape) >= 2:  # For grayscale or RGB format.
            height, width = img_np.shape[:2]
            start_row, start_col = int(height * 0.25), int(width * 0.1)
            end_row, end_col = int(height * 0.75), int(width * 0.9)
            cropped_image = img_np[start_row:end_row, start_col:end_col]
            return cropped_image
        else:
            raise ValueError("Invalid image format")
    except Exception as e:
        print(f"Error fetching image from URL: {e}")
        return None

# Functions to Save Model Results to Files for Satellite Integration
# Function to save the result of the Continuum model to a file.
def save_continuum_result_to_file(img, file_path):  # Saves the result of the Continuum model to the given file.
    # Get the model prediction.
    continuum_result = predict_continuum_image(img)  # Writes the result to the file.
    try:
        with open(file_path, 'w') as file:  # Overwrites the file if it exists; otherwise, creates a new one.
            json.dump(continuum_result, file)
        print(f"Result successfully saved to {file_path}.")
    except Exception as e:
        print(f"An error occurred while writing to the file: {e}")

# Function to save the result of the Magnetogram model to a file.
def save_magnetogram_result_to_file(img, file_path):  # Saves the result of the Magnetogram model to the given file.
    # Get the model prediction.
    magnetogram_result = predict_magnetogram_image(img)
    # Writes the result to the file.
    try:
        with open(file_path, 'w') as file:  # Overwrites the file if it exists; otherwise, creates a new one.
            json.dump(magnetogram_result, file)
        print(f"Result successfully saved to {file_path}.")
    except Exception as e:
        print(f"An error occurred while writing to the file: {e}")

# Definition and Launching of the Interface
# The interface function defines the Gradio user interface.
def interface():
    # Fetch and crop Continuum and Magnetogram images from their URLs.
    continuum_image = fetch_and_crop_image(CONTINUUM_URL)
    magnetogram_image = fetch_and_crop_image(MAGNETOGRAM_URL)
    # Create an interface using Gradio Blocks.
    with gr.Blocks() as demo:
        # The first row contains the images and a field to display the current KP index.
        with gr.Row():
            # Image upload area for the Continuum image.
            continuum_input = gr.Image(type="numpy", label="Continuum Image", value=continuum_image)
            # Image upload area for the Magnetogram image.
            magnetogram_input = gr.Image(type="numpy", label="Magnetogram Image", value=magnetogram_image)
            # Text box to display the current KP index.
            kp_output = gr.Textbox(label="Current KP Index")
        # The second row contains fields for model predictions.
        with gr.Row():
            # Label field for the Continuum model prediction.
            continuum_output = gr.Label(label="Continuum Model Prediction")
            # Label field for the Magnetogram model prediction.
            magnetogram_output = gr.Label(label="Magnetogram Model Prediction")
            # Text box for KP classification.
            kp_classification_output = gr.Textbox(label="KP Classification")
        # A button is defined to trigger the prediction.
        predict_button = gr.Button("Predict")
        # The prediction function takes inputs and returns outputs.
        def predict_all(cont_img, mag_img):  # Synchronously fetches the KP value.
            kp_value = fetch_kp_sync()  # Classifies the KP value.
            kp_classification = classify_kp(kp_value)  # Predicts the Continuum image using the model.
            continuum_result = predict_continuum_image(cont_img)
            # Predicts the Magnetogram image using the model.
            magnetogram_result = predict_magnetogram_image(mag_img)
            # Saves the Continuum result to a file.
            save_continuum_result_to_file(cont_img, "continuum_result.json")
            # Saves the Magnetogram result to a file.
            save_magnetogram_result_to_file(mag_img, "magnetogram_result.json")
            # Returns all results.
            return continuum_result, magnetogram_result, kp_value, kp_classification
        # When the button is clicked, the predict_all function will be executed.
        predict_button.click(
            fn=predict_all,
            inputs=[continuum_input, magnetogram_input],
            outputs=[continuum_output, magnetogram_output, kp_output, kp_classification_output],
        )
    return demo  # Returns the interface.
    
# Launch the interface.
if __name__ == "__main__":
    interface().launch()  # Launches the interface.


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
